In [14]:
import glob
import random
from typing import List
from collections import defaultdict

import numpy as np
from numpy.random import choice

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from symusic import Score
from miditok import REMI, TokenizerConfig
from miditok.pytorch_data import DatasetMIDI, DataCollator

@inproceedings{pop909-ismir2020,
    author = {Ziyu Wang* and Ke Chen* and Junyan Jiang and Yiyi Zhang and Maoran Xu and Shuqi Dai and Guxian Bin and Gus Xia},
    title = {POP909: A Pop-song Dataset for Music Arrangement Generation},
    booktitle = {Proceedings of 21st International Conference on Music Information Retrieval, {ISMIR}},
    year = {2020}
}

In [15]:
FILE_PATH = glob.glob("POP909/**/*.mid", recursive=True)[:300]
TEST_FILE_PATH = glob.glob("POP909/**/*.mid", recursive=True)[:300]

tokenizer = REMI()
train_dataset = DatasetMIDI(
    files_paths=FILE_PATH,
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
    pre_tokenize = True
)
test_dataset = DatasetMIDI(
    files_paths=TEST_FILE_PATH,
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
    pre_tokenize = True
)
collator = DataCollator(tokenizer.pad_token_id)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collator)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collator)

Pre-tokenizing: 100%|███████████████████████████████████████████████████████████████| 300/300 [00:04<00:00, 74.91it/s]


In [16]:
class MusicRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super(MusicRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_dim, 
            num_heads=1,
            batch_first=True
        )
        self.ln1 = nn.LayerNorm(hidden_dim)
        self.ln2 = nn.LayerNorm(hidden_dim)

        self.dropout = nn.Dropout(0.1)
        
        #self.fc = nn.Linear(hidden_dim, vocab_size)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim * 2)
        self.fc2 = nn.Linear(hidden_dim*2, vocab_size)

        self._init_weights()
    
    def _init_weights(self):
        # Initialize attention weights with smaller values
        for name, param in self.attention.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param, gain=0.1)
        
        # Initialize FC layers
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.zeros_(self.fc1.bias)
        nn.init.zeros_(self.fc2.bias)

    def forward(self, x, hidden=None):
        # x: (batch_size, seq_length)
        x = self.embedding(x)  # (batch_size, seq_length, embedding_dim)
        out, hidden = self.rnn(x, hidden)  # out: (batch_size, seq_length, hidden_dim)
        out = self.ln1(out)
        
        attn_out, _ = self.attention(out, out, out)
        out = 0.7 * out + 0.3 * attn_out
        out = self.ln2(out)

        out = F.relu(self.fc1(out))
        out = self.fc2(out)  # (batch_size, seq_length, vocab_size)
        return out, hidden

In [17]:
def train(model, train_loader, val_loader, vocab_size, num_epochs=20, lr=0.0001, device='mps'):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        # --------- Training ---------
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            batch = batch['input_ids'].to(device)  # (batch_size, seq_length)

            inputs = batch[:, :-1]
            targets = batch[:, 1:]

            optimizer.zero_grad()
            outputs, _ = model(inputs)
            outputs = outputs.reshape(-1, vocab_size)
            targets = targets.reshape(-1)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)

        # --------- Validation ---------
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch['input_ids'].to(device)

                inputs = batch[:, :-1]
                targets = batch[:, 1:]

                outputs, _ = model(inputs)
                outputs = outputs.reshape(-1, vocab_size)
                targets = targets.reshape(-1)

                loss = criterion(outputs, targets)
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")


# Example usage
if __name__ == "__main__":
    vocab_size = tokenizer.vocab_size
    embedding_dim = 256
    hidden_dim = 512
    num_layers = 2

    model = MusicRNN(vocab_size, embedding_dim, hidden_dim, num_layers)
    train(model, train_loader, test_loader, vocab_size)

Epoch 1/20 | Train Loss: 3.6736 | Val Loss: 2.6619
Epoch 2/20 | Train Loss: 2.3204 | Val Loss: 2.0995
Epoch 3/20 | Train Loss: 2.0272 | Val Loss: 1.9381
Epoch 4/20 | Train Loss: 1.9059 | Val Loss: 1.8496
Epoch 5/20 | Train Loss: 1.8153 | Val Loss: 1.7682
Epoch 6/20 | Train Loss: 1.7513 | Val Loss: 1.6968
Epoch 7/20 | Train Loss: 1.6864 | Val Loss: 1.6447
Epoch 8/20 | Train Loss: 1.6339 | Val Loss: 1.6007
Epoch 9/20 | Train Loss: 1.5766 | Val Loss: 1.5254
Epoch 10/20 | Train Loss: 1.5054 | Val Loss: 1.4610
Epoch 11/20 | Train Loss: 1.4033 | Val Loss: 1.2828
Epoch 12/20 | Train Loss: 1.0920 | Val Loss: 0.8220
Epoch 13/20 | Train Loss: 0.5548 | Val Loss: 0.2778
Epoch 14/20 | Train Loss: 0.1419 | Val Loss: 0.0639
Epoch 15/20 | Train Loss: 0.0458 | Val Loss: 0.0293
Epoch 16/20 | Train Loss: 0.0236 | Val Loss: 0.0189
Epoch 17/20 | Train Loss: 0.0183 | Val Loss: 0.0150
Epoch 18/20 | Train Loss: 0.0140 | Val Loss: 0.0121
Epoch 19/20 | Train Loss: 0.0109 | Val Loss: 0.0106
Epoch 20/20 | Train L

In [21]:
def sample(model, start_token, max_length=100, temperature=1.0, device='mps'):
    model = model.to(device)
    model.eval()

    generated = [start_token]
    input_token = torch.tensor([[start_token]], device=device)  # (1, 1)

    hidden = None

    for _ in range(max_length):
        output, hidden = model(input_token, hidden)  # output: (1, 1, vocab_size)
        output = output[:, -1, :]  # take the last output
        output = output / temperature  # adjust randomness

        probs = F.softmax(output, dim=-1)  # (1, vocab_size)
        next_token = torch.multinomial(probs, num_samples=1).item()
        generated.append(next_token)
        if next_token == 2 or next_token == 0: # reach end of sequence
          break

        input_token = torch.tensor([[next_token]], device=device)

    return generated

start_token = tokenizer.special_tokens_ids[1]
generated_sequence = sample(model, start_token, max_length=1024)

print("Generated token sequence:")
print(generated_sequence)

Generated token sequence:
[1, 116, 116, 116, 116, 116, 116, 116, 116, 28, 33, 116, 116, 116, 116, 116, 116, 36, 116, 116, 116, 160, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 13, 116, 116, 116, 116, 31, 116, 31, 116, 116, 36, 116, 116, 116, 116, 28, 28, 28, 28, 28, 28, 28, 35, 33, 33, 33, 33, 33, 33, 102, 102, 152, 102, 102, 0]


In [22]:
from midi2audio import FluidSynth # Import library
from IPython.display import Audio, display
fs = FluidSynth("FluidR3Mono_GM.sf3") # Initialize FluidSynth

output_score = tokenizer.tokens_to_midi([generated_sequence])
output_score.dump_midi(f"rnn.mid")
fs.midi_to_audio("rnn.mid", "rnn.wav")

/var/folders/kg/pzsfxj1n6lvby7hkwxtd5nbr0000gn/T/ipykernel_16130/388545212.py:5: UserWarning: miditok: The `tokens_to_midi` method had been renamed `decode`. It is now depreciated and will be removed in future updates.
  output_score = tokenizer.tokens_to_midi([generated_sequence])


FluidSynth runtime version 2.4.5
Copyright (C) 2000-2025 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'rnn.wav'..


In [23]:
display(Audio("rnn.wav"))

In [24]:
token_list = []
for tokens in generated_sequence:
    token_list.append(tokenizer.token_id_type(tokens))
print(token_list)

['BOS', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Pitch', 'Pitch', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Pitch', 'Velocity', 'Velocity', 'Velocity', 'Duration', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Pitch', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Pitch', 'Velocity', 'Pitch', 'Velocity', 'Velocity', 'Pitch', 'Velocity', 'Velocity', 'Velocity', 'Velocity', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Velocity', 'Velocity', 'Duration', 'Velocity', 'Velocity', 'PAD']
